In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
# Creation of the dataframe and Dropping the useless features
to_open = "Corr-20240420_TheBlaze.csv"
output_name = to_open.split('-')[1]
date = output_name.split('_')[0][:4] + "-" + output_name.split('_')[0][4:6] + "-" + output_name.split('_')[0][6:8]
print(date)
df = pd.read_csv(f"weekly_corrected//{to_open}")
df

2024-04-20


,Unnamed: 0,time,number,latitude,longitude,cape,sp,tcw,sshf,slhf,...,skt,cin,sm,st,sd,sf,tcc,tp,mx2t6,mn2t6
0,0,2024-04-20 00:00:00,1,50.75,4.25,0.711205,101015.510,9.117358,-81.0,0.0,...,276.38214,NaN,414.50644,280.75528,0.000000,-4.768372e-07,35.290962,0.000000,NaN,NaN
1,1,2024-04-20 00:00:00,2,50.75,4.25,0.192982,101004.070,9.216370,-81.0,0.0,...,277.33350,NaN,395.47552,281.10068,0.000000,-4.768372e-07,49.094830,0.000000,NaN,NaN
2,2,2024-04-20 00:00:00,3,50.75,4.25,0.237518,100971.210,9.140598,-81.0,0.0,...,277.04877,NaN,402.27286,281.09976,0.000000,-4.768372e-07,33.401817,0.000000,NaN,NaN
3,3,2024-04-20 00:00:00,4,50.75,4.25,1.349537,101007.300,9.734987,-81.0,0.0,...,277.16885,NaN,410.11377,280.82748,0.000000,-4.768372e-07,45.643100,0.000000,NaN,NaN
4,4,2024-04-20 00:00:00,5,50.75,4.25,0.483131,101007.190,8.827965,-81.0,0.0,...,277.06226,NaN,401.90480,281.19490,0.000000,-4.768372e-07,44.584080,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,1445,2024-04-27 00:00:00,46,50.75,4.25,0.000000,100570.140,9.876022,-19640308.0,-42399124.0,...,274.08590,NaN,385.35736,279.69162,0.029241,7.132080e-01,0.003052,21.270836,281.39400,275.03195
1446,1446,2024-04-27 00:00:00,47,50.75,4.25,4.225403,100350.110,11.523244,-21605916.0,-41192624.0,...,275.02158,NaN,368.58750,280.17642,0.001129,1.411467e-01,97.550280,12.800599,282.07556,276.46814
1447,1447,2024-04-27 00:00:00,48,50.75,4.25,0.000000,100764.910,9.771599,-19861326.0,-45134360.0,...,275.07596,NaN,363.09260,280.90756,0.000000,7.347918e-02,0.000000,10.359112,282.77640,277.00060
1448,1448,2024-04-27 00:00:00,49,50.75,4.25,0.906887,99540.270,18.586174,-17474344.0,-41807384.0,...,283.19310,NaN,392.99603,282.69174,0.000000,6.156504e-01,99.999450,19.372528,285.33390,284.23273


In [4]:
df=df[['time','number','t2m']]

In [5]:
df

,time,number,t2m
0,2024-04-20 00:00:00,1,278.977473
1,2024-04-20 00:00:00,2,279.455910
2,2024-04-20 00:00:00,3,279.581667
3,2024-04-20 00:00:00,4,279.294324
4,2024-04-20 00:00:00,5,279.197717
...,...,...,...
1445,2024-04-27 00:00:00,46,277.234177
1446,2024-04-27 00:00:00,47,277.982261
1447,2024-04-27 00:00:00,48,279.291764
1448,2024-04-27 00:00:00,49,285.388301


In [6]:
df['t2m']=df['t2m']-273.15

/var/folders/lw/bnynzkxj0cq2wnbmqw0rvyzh0000gn/T/ipykernel_34961/836502028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t2m']=df['t2m']-273.15


In [7]:
# Creation of the results Dataframes which will be made into one df later on
result_t2m  = pd.DataFrame(columns=['forecast_date','target','horizon','q0.025','q0.25','q0.5','q0.75','q0.975'])

In [8]:
times=df.time.unique()
number_of_hours=0
#defined earlier
#date = "2023-03-15"
row_index=0
for t in times:
  df_=df[df['time']==t]
  t2m = []
  t2m=df_.t2m.values


  horizon = str(number_of_hours)+' hour'
  result_t2m.loc[row_index]  = [date, 't2m', horizon , np.quantile(t2m,0.025),np.quantile(t2m,0.25),np.quantile(t2m,0.5),np.quantile(t2m,0.75),np.quantile(t2m,0.975)]
  row_index=row_index+1
  number_of_hours = number_of_hours + 6


In [9]:
# Dataframes concatination and result exportation
result_t2m = pd.DataFrame.to_csv(result_t2m,f"weekly_corrected/{output_name}", index=False)

In [10]:
result_t2m